In [1]:


import pandas as pd
import numpy as np

import os

# Obtendo o diretório atual do notebook
diretorio_atual = os.getcwd()

# Definindo o diretório raiz como "arquivos"
diretorio_raiz = r"c:\Users\claudion\OneDrive - Cielo\02. Maritalk"

# Mudando o diretório de trabalho para a pasta raiz
os.chdir(diretorio_raiz)

print(os.getcwd())


c:\Users\claudion\OneDrive - Cielo\02. Maritalk


In [2]:
#Criar função para retornar as noticias mais recentes de uma query
from GoogleNews import GoogleNews

def get_news(query):
    googlenews = GoogleNews()
    googlenews.setlang('pt')
    googlenews.setperiod('7d')
    googlenews.search(query)
    result = googlenews.result()
    df = pd.DataFrame(result)
    return df

df = get_news('RS')

In [7]:
df.columns

Index(['title', 'media', 'date', 'datetime', 'desc', 'link', 'img'], dtype='object')

In [3]:
df = df.head()

In [6]:
import requests # Required to make HTTP requests
from bs4 import BeautifulSoup # Required to parse HTML
import numpy as np # Required to dedupe sites
from urllib.parse import unquote # Required to unquote URLs
query = "chuvas RS"
response = requests.get(f"https://www.google.com/search?q={query}&tbm=nws") # Make the request
soup = BeautifulSoup(response.text, "html.parser") # Parse the HTML
links = soup.find_all("a") # Find all the links in the HTML

# loop over `links` and keep only the one that have the href starting with "/url?q="
urls = []
for l in [link for link in links if link["href"].startswith("/url?q=")]:
    # get the url
    url = l["href"]
    # remove the "/url?q=" part
    url = url.replace("/url?q=", "")
    # remove the part after the "&sa=..."
    url = unquote(url.split("&sa=")[0])
    # special case for google scholar
    if url.startswith("https://scholar.google.com/scholar_url?url=http"):
        url = url.replace("https://scholar.google.com/scholar_url?url=", "").split("&")[0]
    elif 'google.com/' in url: # skip google links
        continue
    if url.endswith('.pdf'): # skip pdf links
        continue
    if '#' in url: # remove anchors (e.g. wikipedia.com/bob#history and wikipedia.com/bob#genetics are the same page)
        url = url.split('#')[0]
    # print the url
    urls.append(url)

# Use numpy to dedupe the list of urls after removing anchors
urls = list(np.unique(urls))[1:5]
urls

['https://agenciabrasil.ebc.com.br/geral/noticia/2024-05/quinze-quilombos-estao-totalmente-isolados-pelas-chuvas-no-rs',
 'https://g1.globo.com/meio-ambiente/noticia/2024/05/16/apos-passagem-de-frente-fria-chuva-volta-ao-rs-nesta-quinta-volumes-nao-devem-superar-100-milimetros.ghtml',
 'https://g1.globo.com/politica/noticia/2024/05/15/lula-anuncia-medidas-para-familias-atingidas-pelas-enxurradas-no-rs.ghtml',
 'https://jovempan.com.br/noticias/brasil/quase-12-mil-animais-ja-foram-resgatados-das-enchentes-no-rs.html']

In [7]:
from readabilipy import simple_json_from_html_string
from langchain.schema import Document # Required to create a Document object
import requests
def scrape_and_parse(url: str) -> Document:
    """Scrape a webpage and parse it into a Document object"""
    req = requests.get(url)
    article = simple_json_from_html_string(req.text, use_readability=True)
    # The following line seems to work with the package versions on my local machine, but not on Google Colab
    # return Document(page_content=article['plain_text'][0]['text'], metadata={'source': url, 'page_title': article['title']})
    return Document(page_content='\n\n'.join([a['text'] for a in article['plain_text']]), metadata={'source': url, 'page_title': article['title']})

In [8]:
dcs = scrape_and_parse(urls[-1])
dcs.page_content

'Quase 12 mil animais já foram resgatados das enchentes no RS | Jovem Pan\n\nMinha conta\n\nJovem Pan\n\n> Notícias\n\n> Brasil\n\n> Quase 12 mil animais já foram resgatados das enchentes no RS\n\nQuase 12 mil animais já foram resgatados das enchentes no RS\n\nONGs e voluntários estão na linha de frente desde o início das chuvas para salvar os bichos\n\n* Por Tamyres Sbrile, * 16/05/2024 20h08,\n\nWILTON JUNIOR/ESTADÃO CONTEÚDOEstado tem enfrentado chuvas fortes desde o fim de abril\n\nSegundo último boletim da Defesa Civil do Rio Grande do Sul, cerca de 11.932 animais foram resgatados das enchentes que atingem o estado. Cães, gatos, porcos, cavalos, como foi o caso do “Caramelo”, e aves estão sendo resgatados diariamente por ONGs e bombeiros civis. Segundo a secretaria estadual do Meio Ambiente e Infraestrutura (Sema), a maioria dos bichos resgatados são cães e gatos, mas animais silvestres também estão sendo socorridos, como guaxinins. Ainda conforme a secretaria, os animais que são 

In [9]:
from utils.llm import *

apikey = read_key()
prompt = f"Faça um resumo da seguinte notícia: {dcs.page_content}"
retorno = get_response(prompt, apikey)
print(retorno)

 A notícia relata o esforço de ONGs e voluntários no resgate de animais afetados pelas enchentes no Rio Grande do Sul. Até a data do último boletim da Defesa Civil, cerca de 11.932 animais, incluindo cães, gatos, porcos, cavalos e aves, foram salvos. A secretaria estadual do Meio Ambiente e Infraestrutura (Sema) informa que a maioria dos animais resgatados são cães e gatos, mas também há animais silvestres, como guaxinins. Os animais passam por uma triagem veterinária e são encaminhados para abrigos ou clínicas, dependendo de sua condição de saúde.

Além disso, o grupo Carrefour Brasil cedeu um galpão em São Leopoldo para servir como abrigo temporário para os animais resgatados. Nas redes sociais, diversos perfis compartilham a rotina dos resgates e incentivam doações e ações para encontrar animais perdidos.

A ONG Campo Bom Pra Cachorro, de Campo Bom, é um exemplo de organização que está atuando no resgate dos animais desde o início das chuvas. Eles criaram um perfil nas redes sociais

In [11]:
#Para cada link, acessar o site e extrair o texto da notícia
import requests
from bs4 import BeautifulSoup

def get_text(link):
    try:
        page = requests.get(link)
        soup = BeautifulSoup(page.text, 'html.parser')
        text = soup.find_all('a')
        for link in text:
            href = link.get('href')
        return text
    except:
        return None

df['text'] = df['link'].apply(get_text)

c:\Users\claudion\OneDrive - Cielo\02. Maritalk\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnnbrasil.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\claudion\OneDrive - Cielo\02. Maritalk\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnnbrasil.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\claudion\OneDrive - Cielo\02. Maritalk\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnnbrasil.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-

In [14]:
df['text'].values[1]

[<p><a href="https://go.microsoft.com/fwlink/?LinkID=62293&amp;IIS70Error=404,0,0x80070002,17763">View more information »</a></p>]

In [ ]:
#https://ealexbarros.medium.com/utilizando-python-para-armazenar-not%C3%ADcias-google-news-api-6d37f84cd426

In [10]:
enderecos = pd.read_excel(r"C:\Users\claudion\Downloads\FDEX_AUSENTES.xlsx")
enderecos

,Completo END,Código SAP
0,"AV JOSE RAIMUNDO,3949 -GRANJAS VAGALUME ,IPAT...",5832
1,"AV. ANEL DO CONTORNO,196 -ESPÍRITO SANTO ,VIT...",4500
2,"R PROF DARTAGNAN ALVES,48 -CENTRO ,ARCOVERDE, PE",5650
3,"R JOAQUIM TAVORA,244 -SAO FRANCISCO ,CARUARU, PE",5500
4,"R FELIPE CAMARAO,235 QUADRA AREA LOTE 02JARDI...",9916
...,...,...
68,"R FENANDO DE NORONHA,4176 -NOVA FLORESTA ,POR...",9867
69,"R JOANA BATISTA HOLANDA,135 ASALGADINHO ,JUAZ...",4152
70,"R FLODOALDO RODRIGUES,665 JARDIM AUREA ,VARGI...",5430
71,"R SEBASTIANA MARIA DE JESUS,317 -NOSSA SENHOR...",7221


In [12]:
import requests


latitudes = []
longitudes = []

for i in range(len(enderecos)):
    countryRegion = 'BR'
    # addressLine = "AV. ANEL DO CONTORNO,196  -ESPÍRITO SANTO ,VITORIA DA CONQUISTA, BA"
    addressLine = enderecos['Completo END'][i]
    maxResults = 1
    BingMapsKey = 'AlCTh-7wnjGDButsudsD62gzEhUfWYVzEowrj4E_XRTGWCM8GpWCjw7En119U76a'
    url = f"http://dev.virtualearth.net/REST/v1/Locations?countryRegion={countryRegion}&addressLine={addressLine}&maxResults={maxResults}&key={BingMapsKey}"

    response = requests.get(url)
    data = response.json()
    status_code = response.status_code

    if status_code != 200:
        print("Erro na requisição")
        print('\n')
        print(i)
        latitude = np.nan
        longitude = np.nan

    else:
        latitude = data['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates'][0]
        longitude = data['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates'][1]
    
    latitudes.append(latitude)
    longitudes.append(longitude)

enderecos['LAT'] = latitudes
enderecos['LONG'] = longitudes

In [14]:
enderecos.to_excel(r"C:\Users\claudion\Downloads\FDEX_AUSENTES.xlsx", index=False)